# Микродиахроническое исследование русских приставок методами дистрибутивной семантики
## Автор: Елизавета Клыкова, БКЛ181
### Часть 6: анализ частотности приставок и глаголов
1. В каждом периоде составить список глаголов для каждой приставки
2. Отранжировать глаголы по частотности
3. Сравнить списки по периодам с использованием расстояния Левенштейна

**В тексте отметить, что порог по частотности = 5 для каждого периода, т.к. ручная разметка выполнена только для глаголов, соответствующих этому условию**

#### Импорт модулей

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import pandas as pd
from tqdm.auto import tqdm
from textdistance import levenshtein

#### Получение списка глаголов

In [3]:
pref_df = pd.read_csv('prefixes_and_lemmas.tsv', sep='\t')
pref_df

,prefix,lemma,freq,freq_0,freq_1,freq_2
0,бе[зс],бездействовать,541,155,255,131
1,бе[зс],бездельничать,308,41,169,98
2,бе[зс],безмолвствовать,726,361,191,174
3,бе[зс],безобразить,149,99,43,7
4,бе[зс],безобразничать,306,110,136,60
...,...,...,...,...,...,...
8440,у,ущемляться,62,8,16,38
8441,у,ущипывать,510,147,222,141
8442,у,уязвлять,1305,606,411,288
8443,у,уяснять,1803,677,740,386


In [4]:
# list of dicts
pref_dict = pref_df.to_dict(orient='records')

#### Переводим частотности в абсолютные

In [5]:
presov_vol = 72199757
sov_vol = 93031557
postsov_vol = 81392744
total_vol = 246624058

In [6]:
for verb in pref_dict:
    verb['abs_freq'] = verb['freq']
    verb['abs_freq0'] = verb['freq_0']
    verb['abs_freq1'] = verb['freq_1']
    verb['abs_freq2'] = verb['freq_2']
    del verb['freq']
    del verb['freq_0']
    del verb['freq_1']
    del verb['freq_2']
    verb['freq'] = round(verb['abs_freq'] / total_vol * 1000000, 3)
    verb['freq0'] = round(verb['abs_freq0'] / presov_vol * 1000000, 3)
    verb['freq1'] = round(verb['abs_freq1'] / sov_vol * 1000000, 3)
    verb['freq2'] = round(verb['abs_freq2'] / postsov_vol * 1000000, 3)

In [7]:
pref_df_new = pd.DataFrame(pref_dict)
pref_df_new

,prefix,lemma,abs_freq,abs_freq0,abs_freq1,abs_freq2,freq,freq0,freq1,freq2
0,бе[зс],бездействовать,541,155,255,131,2.194,2.147,2.741,1.609
1,бе[зс],бездельничать,308,41,169,98,1.249,0.568,1.817,1.204
2,бе[зс],безмолвствовать,726,361,191,174,2.944,5.000,2.053,2.138
3,бе[зс],безобразить,149,99,43,7,0.604,1.371,0.462,0.086
4,бе[зс],безобразничать,306,110,136,60,1.241,1.524,1.462,0.737
...,...,...,...,...,...,...,...,...,...,...
8440,у,ущемляться,62,8,16,38,0.251,0.111,0.172,0.467
8441,у,ущипывать,510,147,222,141,2.068,2.036,2.386,1.732
8442,у,уязвлять,1305,606,411,288,5.291,8.393,4.418,3.538
8443,у,уяснять,1803,677,740,386,7.311,9.377,7.954,4.742


In [8]:
pref_df_new.to_csv('prefixes_and_lemmas_freq.tsv', sep='\t', index=False)

#### Получаем ранжированные списки

In [9]:
prefixes = sorted(list(set([verb['prefix'] for verb in pref_dict])))

In [12]:
prefs_with_verbs = {pref: [verb for verb in pref_dict
                           if verb['prefix'] == pref]
                    for pref in prefixes}

In [13]:
presov_ranged = {pref: [verb['lemma']
                        for verb in sorted(prefs_with_verbs[pref],
                                           key=lambda x: x['freq0'],
                                           reverse=True)]
                 for pref in prefs_with_verbs}

In [14]:
sov_ranged = {pref: [verb['lemma']
                     for verb in sorted(prefs_with_verbs[pref],
                                        key=lambda x: x['freq1'],
                                        reverse=True)]
              for pref in prefs_with_verbs}

In [15]:
postsov_ranged = {pref: [verb['lemma']
                         for verb in sorted(prefs_with_verbs[pref],
                                            key=lambda x: x['freq2'],
                                            reverse=True)]
                  for pref in prefs_with_verbs}

#### Подсчет расстояния Левенштейна

In [16]:
levst_dict = {pref: {'pre_to_sov': levenshtein.distance(presov_ranged[pref],
                                                        sov_ranged[pref]),
                     'sov_to_post': levenshtein.distance(sov_ranged[pref],
                                                         postsov_ranged[pref]),
                     'pre_to_post': levenshtein.distance(presov_ranged[pref],
                                                         postsov_ranged[pref])}
              for pref in tqdm(prefixes)}

  0%|          | 0/26 [00:00<?, ?it/s]

In [17]:
pref_stats = pref_df.groupby(['prefix']).size().to_dict()

In [18]:
stats_for_df = []
for pref in pref_stats:
    stats_for_df.append({'prefix': pref,
                         'verbs': pref_stats[pref],
                         'pre_to_sov': levst_dict[pref]['pre_to_sov'],
                         'sov_to_post': levst_dict[pref]['sov_to_post'],
                         'pre_to_post': levst_dict[pref]['pre_to_post']})

In [19]:
stats_df = pd.DataFrame(stats_for_df)
stats_df

,prefix,verbs,pre_to_sov,sov_to_post,pre_to_post
0,бе[зс],11,6,6,6
1,в[зс],150,137,132,137
2,во?,137,128,120,127
3,во[зс],122,111,105,113
4,вы,444,429,415,433
5,до,153,140,135,141
6,за,937,915,909,924
7,и[зс],276,265,257,262
8,на,524,507,496,510
9,над,17,12,12,13
